# 서울시 자치구 서대문구에서 치킨집이 가장 많은 동은 어디인가?

### 자료 출처
- 데이터 다운로드 : LOCALDATA 공공데이터
- https://www.localdata.kr/

## 패키지 설정

In [3]:
# 패키지 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# matplotlib 한글 세팅

import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

## 데이터 가져오기

In [5]:
data = pd.read_csv('./서울특별시_일반음식점.csv',
                  encoding='cp949',
                  low_memory=False)

In [7]:
data.head(2)

,번호,개방서비스명,개방서비스아이디,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
0,1,일반음식점,07_24_04_P,3150000,3150000-101-2020-00448,20200803,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,N,35.19,NaN,NaN,NaN,NaN
1,2,일반음식점,07_24_04_P,3150000,3150000-101-2020-00449,20200803,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,N,50.37,NaN,NaN,NaN,NaN


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472915 entries, 0 to 472914
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   번호           472915 non-null  int64  
 1   개방서비스명       472915 non-null  object 
 2   개방서비스아이디     472915 non-null  object 
 3   개방자치단체코드     472915 non-null  int64  
 4   관리번호         472915 non-null  object 
 5   인허가일자        472915 non-null  int64  
 6   인허가취소일자      0 non-null       float64
 7   영업상태구분코드     472915 non-null  int64  
 8   영업상태명        472915 non-null  object 
 9   상세영업상태코드     472915 non-null  int64  
 10  상세영업상태명      472915 non-null  object 
 11  폐업일자         348707 non-null  float64
 12  휴업시작일자       0 non-null       float64
 13  휴업종료일자       0 non-null       float64
 14  재개업일자        0 non-null       float64
 15  소재지전화        327325 non-null  object 
 16  소재지면적        439015 non-null  object 
 17  소재지우편번호      472670 non-null  float64
 18  소재지전체주소      472678 non-

## 데이터 처리

- 문제 : 서울시 자치구 서대문구에서 치킨집이 가장 많은 동은 어디인가?
- 변수 확인 :
  1. 음식점이면서, 치킨과 관련된 항목을 취급해야 함
  2. 현재 영업중이어야 함  
  3. 서대문구에 위치해야 함

#### 1. 음식점이면서, 치킨과 관련된 항목을 취급해야 함

* Column을 보았을 때, 1)업태구분명 2)위생업태명 으로 판단 가능해 보임
* 1)업태구분명 2)위생업태명의 데이터 확인

In [17]:
# 1) 업태구분명 데이터 확인
set(data.업태구분명.values)

{'193959.150482967',
 nan,
 '간이주점',
 '감성주점',
 '경양식',
 '기타',
 '기타 휴게음식점',
 '김밥(도시락)',
 '까페',
 '냉면집',
 '다방',
 '라이브카페',
 '룸살롱',
 '복어취급',
 '분식',
 '뷔페식',
 '식육(숯불구이)',
 '식품등 수입판매업',
 '식품소분업',
 '외국음식전문점(인도,태국등)',
 '이동조리',
 '일반조리판매',
 '일식',
 '전통찻집',
 '정종/대포집/소주방',
 '제과점영업',
 '중국식',
 '출장조리',
 '커피숍',
 '키즈카페',
 '탕류(보신용)',
 '통닭(치킨)',
 '패밀리레스트랑',
 '패스트푸드',
 '한식',
 '호프/통닭',
 '횟집'}

In [16]:
# 2) 위생업태명 데이터 확인
set(data.위생업태명.values)

{nan,
 '간이주점',
 '감성주점',
 '경양식',
 '기타',
 '기타 휴게음식점',
 '김밥(도시락)',
 '까페',
 '냉면집',
 '다방',
 '라이브카페',
 '룸살롱',
 '복어취급',
 '분식',
 '뷔페식',
 '식육(숯불구이)',
 '식품등 수입판매업',
 '식품소분업',
 '외국음식전문점(인도,태국등)',
 '이동조리',
 '일반조리판매',
 '일식',
 '전통찻집',
 '정종/대포집/소주방',
 '제과점영업',
 '중국식',
 '출장조리',
 '커피숍',
 '키즈카페',
 '탕류(보신용)',
 '통닭(치킨)',
 '패밀리레스트랑',
 '패스트푸드',
 '한식',
 '호프/통닭',
 '횟집'}

* 동일한 데이터로 판단하여, 위생업태명 데이터로 판단
* 위생업태명에서 '음식점이면서, 치킨과 관련된' 항목은 :
  - 통닭(치킨)
  - 호프/통닭

* 위 두 항목에서 추출한 keywords '통닭', '치킨'을 활용하기 위하여 재점검
  - 조건 인덱싱 (Boolean Indexing) 을 사용하여 위생업태명 중 '통닭', '치킨' 이 포함된 업태명 반환 

In [18]:
# 조건 인덱싱 (Boolean Indexing) 을 사용하여 위생업태명 중 '통닭', '치킨' 이 포함된 업태명 반환
set(data.위생업태명[data.위생업태명.str.contains('통닭|치킨', na=False)])

{'통닭(치킨)', '호프/통닭'}

#### 2. 현재 영업중이어야 함 